# Random Forest For VQA

## Init Dataset

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys, pathlib
sys.path.append("..")
import pandas as pd
root_dir = "/home/hongyu/Visual-RAG-LLaVA-Med"
lesion_balanced_excel_file = root_dir + "/data/lesion balanced dataset_new_20241210.xlsx"
multi_modal_vqa_excel_file = root_dir + "/data/Multimodal VQA Dataset/Multimodal VQA dataset_1015.xlsx"
lesion_balanced_df = pd.read_excel(lesion_balanced_excel_file)
multi_modal_vqa_df = pd.read_excel(multi_modal_vqa_excel_file)
lesion_balanced_df.describe()
multi_modal_vqa_df.describe()

,imid,folder_standardized,Finding,modality,caption,impath,qtype,Q,A
count,85572,75572,68361,75572,75572,85572,85572,85572,85572
unique,13403,641,459,3,672,13403,7,5173,4548
top,DeepDRiD_99_l1,wet age-related macular degeneration,"abnormal macula, choroidal neovascularization",color fundus,"color fundus,Normal",/home/danli/data/public/alldatasets/DeepDRiD_9...,open-ended,What description would you give this image?,Right eye.
freq,36,3368,3368,45196,1900,36,47424,1164,5000


,Diagnosis,Case number,caption,Q,A
count,1830,1830,1830,1830,1830
unique,13,208,158,859,1155
top,central retinal vein occlusion,AMD_13,"color fundus, abnormal optic disc, enlarged op...",What is the diagnosis?,This is a color fundus image.
freq,156,17,42,39,208


## Left Right Eye

In [9]:
# filter data for left and right eye
left_right_eye_lesion_balanced_df = lesion_balanced_df[
    (lesion_balanced_df["A"] == "Left eye.") | (lesion_balanced_df["A"] == "Right eye.")
]
# left_right_eye_vqa_df = multi_modal_vqa_df[
#     (multi_modal_vqa_df["A"] == "Left eye.") | (multi_modal_vqa_df["A"] == "Right eye.")
# ]
left_right_eye_lesion_balanced_df.drop_duplicates(subset="impath", keep="first")
left_right_eye_lesion_balanced_df.describe()

,imid,folder_standardized,Finding,modality,caption,impath,qtype,Q,A
0,imgbank_536,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/imgbank_53...,laterality,Which eye is featured in this photograph?,Right eye.
1,imgbank_30016,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/imgbank_30...,laterality,Which eye is shown in this picture?,Right eye.
2,imgbank_8367,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/imgbank_83...,laterality,Which eye is illustrated in this image?,Right eye.
3,imgbank_10069,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/imgbank_10...,laterality,What eye does this picture refer to?,Right eye.
4,imgbank_5868,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/imgbank_58...,laterality,What eye does this picture refer to?,Right eye.
...,...,...,...,...,...,...,...,...,...
9995,APTOS2019_ee1ec90b980f,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/APTOS2019_...,laterality,Is this an image of the left eye or the right ...,Left eye.
9996,AIROGS_TRAIN066373,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/AIROGS_TRA...,laterality,Is this image of the left or right eye?,Left eye.
9997,EyePACs_10384_left,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/EyePACs_10...,laterality,Is this an image of the left eye or the right ...,Left eye.
9998,EyePACs_35327_right,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/EyePACs_35...,laterality,Which eye is shown in this picture?,Left eye.


,imid,folder_standardized,Finding,modality,caption,impath,qtype,Q,A
count,10000,0,0,0,0,10000,10000,10000,10000
unique,10000,0,0,0,0,10000,1,10,2
top,imgbank_536,NaN,NaN,NaN,NaN,/home/danli/data/public/alldatasets/imgbank_53...,laterality,Which eye is illustrated in this image?,Right eye.
freq,1,NaN,NaN,NaN,NaN,1,10000,1038,5000


In [12]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split


def load_image(img_path, size=(128, 128)):
    """Load an image from a path and resize it."""
    img = Image.open(img_path).convert("L")  # 转换为灰度图
    img = img.resize(size)  # 调整大小
    return np.array(img).flatten()  # 将图片展平为一维数组


left_right_eye_lesion_balanced_df["impath"] = left_right_eye_lesion_balanced_df[
    "impath"
].apply(
    lambda x: x.replace(
        "/home/danli/data/public/alldatasets", "/home/hongyu/alldataset/images"
    )
)
left_right_eye_lesion_balanced_df["features"] = left_right_eye_lesion_balanced_df[
    "impath"
].apply(load_image)

X = np.vstack(left_right_eye_lesion_balanced_df["features"].values)
y = left_right_eye_lesion_balanced_df["A"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

/tmp/ipykernel_2585108/1246619598.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_right_eye_lesion_balanced_df["impath"] = left_right_eye_lesion_balanced_df[


/tmp/ipykernel_2585108/1246619598.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_right_eye_lesion_balanced_df["features"] = left_right_eye_lesion_balanced_df[


In [13]:
from sklearn.ensemble import RandomForestClassifier
# 创建随机森林分类器实例
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
clf.fit(X_train, y_train)

# 测试模型准确率
accuracy = clf.score(X_test, y_test)
print(f"模型准确率: {accuracy * 100:.2f}%")

RandomForestClassifier(random_state=42)

模型准确率: 84.15%
